In [ ]:
pip install torchtext==0.6.0

In [2]:
import torch
from torchtext import data
import spacy

SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

nlp = spacy.load("en_core_web_sm")

TEXT = data.Field(tokenize=nlp)
LABEL = data.LabelField(dtype=torch.float)

In [3]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:15<00:00, 5.61MB/s]


In [4]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [5]:
print(vars(train_data.examples[0]))
print(vars(test_data.examples[24999]))

{'text': ['The', 'greatest', 'games', 'of', 'Kasparov', 'or', 'Fischer', 'can', 'be', 'a', 'mess', 'for', 'a', 'total', 'rookie.', 'This', 'is', 'a', 'great', 'movie.', 'There', 'is', 'no', 'special', 'agency', 'involved', 'in', 'the', 'plot.', 'This', 'is', 'the', 'clue!', 'This', 'is', 'a', 'PRIVATE', 'plot,', 'built', 'as', 'a', 'PRIVATE', 'enterprise.', 'This', 'is', 'a', 'self-destructive', 'and', 'a', 'self', 'organized', 'plot.', 'As', 'a', 'conclusion,', 'the', 'scenario', 'described', 'the', 'perfect', 'professional', 'plot:', 'private,', 'self', '\x96organized,', 'self-destructive,', 'with', 'no', 'trace', 'at', 'the', 'end.', 'Anyone', 'can', 'be', 'behind', 'the', 'plot:', 'a', 'smart', '"director"', 'with', 'some', 'money.', 'All', 'can', 'be', 'done', 'just', 'by', 'delegation.', 'The', '"director"', 'must', 'be', 'just', 'trigger.', 'If', 'the', 'normal', 'viewer', 'cannot', 'see', 'the', 'essence', 'of', 'the', 'plot', 'in', 'the', 'explicit', 'sequences', 'of', 'the', 

In [6]:
import random

train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [7]:
print(f'Number of training examples: {len(train_data)}') # 70%
print(f'Number of validation examples: {len(valid_data)}') # 30%
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [8]:
TEXT.build_vocab(train_data, max_size=25000)
LABEL.build_vocab(train_data)

In [9]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [10]:
print(TEXT.vocab.freqs.most_common(20))
print(TEXT.vocab.itos[:10])

[('the', 200772), ('a', 108599), ('and', 107006), ('of', 99709), ('to', 93141), ('is', 72139), ('in', 60034), ('I', 46333), ('that', 45050), ('this', 39891), ('it', 38246), ('/><br', 35635), ('was', 32910), ('as', 29773), ('with', 29100), ('for', 28816), ('The', 23786), ('but', 23763), ('on', 21650), ('movie', 21246)]
['<unk>', '<pad>', 'the', 'a', 'and', 'of', 'to', 'is', 'in', 'I']


In [11]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [12]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device)

In [14]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):

        embedded = self.embedding(text)

        output, hidden = self.rnn(embedded)

        assert torch.equal(output[-1,:,:], hidden.squeeze(0))

        return self.fc(hidden.squeeze(0))

In [15]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [24]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [25]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum()/len(correct)
    return acc

In [26]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:

        optimizer.zero_grad()

        predictions = model(batch.text).squeeze(1)

        loss = criterion(predictions, batch.label)

        acc = binary_accuracy(predictions, batch.label)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [27]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:

            predictions = model(batch.text).squeeze(1)

            loss = criterion(predictions, batch.label)

            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [28]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')

| Epoch: 01 | Train Loss: 0.693 | Train Acc: 50.01% | Val. Loss: 0.697 | Val. Acc: 48.98% |
| Epoch: 02 | Train Loss: 0.693 | Train Acc: 49.84% | Val. Loss: 0.697 | Val. Acc: 49.22% |
| Epoch: 03 | Train Loss: 0.693 | Train Acc: 50.19% | Val. Loss: 0.697 | Val. Acc: 49.25% |
| Epoch: 04 | Train Loss: 0.693 | Train Acc: 50.37% | Val. Loss: 0.697 | Val. Acc: 49.54% |
| Epoch: 05 | Train Loss: 0.693 | Train Acc: 50.66% | Val. Loss: 0.697 | Val. Acc: 51.51% |


In [30]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% |')

| Test Loss: 0.714 | Test Acc: 44.32% |


In [42]:
def predict_sentiment(sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [43]:
print(predict_sentiment("This film is terrible"))

0.4209724962711334


In [44]:
print(predict_sentiment("This film is great"))

0.5367135405540466


In [45]:
# torch.save(model, "model.pth")